In [37]:
#Import important Libraries
import numpy as np
import pandas as pd
%pylab inline
import seaborn as sns
import matplotlib.pyplot as plt
import re
# Importing Neccessary Libraries
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
import os
from pandas.plotting import scatter_matrix

Populating the interactive namespace from numpy and matplotlib


In [38]:
path = os.getcwd()

In [39]:
# Importing Dataset
df_ajah_sale = pd.read_csv(path + '/Areas/ajah_sale_clean.csv')
df_gbagada_sale = pd.read_csv(path + '/Areas/gbagada_sale_clean.csv')
df_ikeja_sale = pd.read_csv(path + '/Areas/ikeja_sale_clean.csv')
df_ikorodu_sale = pd.read_csv(path + '/Areas/ikorodu_sale_clean.csv')
df_ikoyi_sale = pd.read_csv(path + '/Areas/ikoyi_sale_clean.csv')
df_lekki_sale = pd.read_csv(path + '/Areas/lekki_sale_clean.csv')
df_surulere_sale = pd.read_csv(path + '/Areas/surulere_sale_clean.csv')
df_yaba_sale = pd.read_csv(path + '/Areas/yaba_sale_clean.csv')

In [40]:
df_sale = pd.concat([df_ajah_sale, df_gbagada_sale, df_ikeja_sale, df_ikorodu_sale, df_ikoyi_sale, df_lekki_sale, df_surulere_sale, df_yaba_sale], join='outer', ignore_index=True)

In [41]:
df_sale.drop('Unnamed: 0', 1, inplace=True)
df_sale.head(3)

,Property,Location,Description,Features,Price,Bed,Bath,Toilet,Built or Serviced,Property Type,Area,Purchase Type,Parking Space,Security,Electricity,Furnished,Security Doors,CCTV
0,1 bedroom mini flat flat / apartment for sale,off general paints bus stop abraham adesanya e...,pacific manor ii is a residential estate with ...,1 bedrooms,17500000.0,1.0,1.0,2.0,NaN,Mini flat,ajah,sale,0,0,0,0,0,1
1,4 bedroom terraced duplex house for sale,ajahajah lagos,nicely buit 4 bedroom terrace duplex;\n\nlocat...,4 bedrooms\nnew,33000000.0,4.0,4.0,5.0,Newly Built,Terraced duplex,ajah,sale,0,1,0,0,0,0
2,4 bedroom terraced duplex house for sale,ekki palm cityajah lagos,brand new 4 bedroom terrace duplex;\n\nlocatio...,4 bedrooms\nnew,38000000.0,4.0,4.0,5.0,Newly Built,Terraced duplex,ajah,sale,0,1,0,0,0,0


In [42]:
         ### CREATE NEW COLUMNS FOR HOUSES ATRIBUTES THAT MAY AFFECT THE PRICE ###
## BOY QUATERS ##
df_sale['BQ'] = np.where(((df_sale.Description.str.contains('bq'))|(df_sale.Features.str.contains('bq'))), 1, 0)

## GYM ##
df_sale['Gym'] = np.where(((df_sale.Description.str.contains('gym'))|(df_sale.Features.str.contains('gym'))), 1, 0)

## POOL ##
df_sale['Pool'] = np.where(((df_sale.Description.str.contains('pool'))|(df_sale.Features.str.contains('pool'))), 1, 0)

# Basic EDA

In [7]:
print("The number of rows and colums are {} and also called shape of the matrix".format(df_sale.shape))

The number of rows and colums are (25683, 21) and also called shape of the matrix


In [8]:
print("Columns names are \n {}".format(df_sale.columns))

Columns names are 
 Index(['Property', 'Location', 'Description', 'Features', 'Price', 'Bed',
       'Bath', 'Toilet', 'Built or Serviced', 'Property Type', 'Area',
       'Purchase Type', 'Parking Space', 'Security', 'Electricity',
       'Furnished', 'Security Doors', 'CCTV', 'BQ', 'Gym', 'Pool'],
      dtype='object')


In [9]:
df_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25683 entries, 0 to 25682
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Property           25683 non-null  object 
 1   Location           25683 non-null  object 
 2   Description        25679 non-null  object 
 3   Features           24444 non-null  object 
 4   Price              25683 non-null  float64
 5   Bed                24572 non-null  float64
 6   Bath               23576 non-null  float64
 7   Toilet             23271 non-null  float64
 8   Built or Serviced  14925 non-null  object 
 9   Property Type      25683 non-null  object 
 10  Area               25683 non-null  object 
 11  Purchase Type      25683 non-null  object 
 12  Parking Space      25683 non-null  int64  
 13  Security           25683 non-null  int64  
 14  Electricity        25683 non-null  int64  
 15  Furnished          25683 non-null  int64  
 16  Security Doors     256

In [10]:
pd.crosstab(df_sale['Bed'],df_sale['Area'],values=df_sale['Price'], aggfunc='count' )

Area,ajah,gbagada,ikeja,ikorodu,ikoyi,lekki,surulere,yaba
Bed,,,,,,,,
0.0,108.0,29.0,61.0,54.0,72.0,328.0,41.0,37.0
1.0,111.0,20.0,6.0,31.0,22.0,112.0,23.0,41.0
2.0,353.0,24.0,59.0,128.0,103.0,672.0,69.0,98.0
3.0,1072.0,193.0,413.0,317.0,623.0,1497.0,332.0,330.0
4.0,1465.0,175.0,612.0,141.0,839.0,6916.0,142.0,106.0
5.0,529.0,79.0,430.0,53.0,789.0,4147.0,30.0,16.0
6.0,52.0,21.0,51.0,29.0,82.0,168.0,8.0,5.0
7.0,8.0,3.0,19.0,9.0,42.0,30.0,2.0,2.0
8.0,5.0,NaN,4.0,13.0,16.0,7.0,2.0,2.0


In [43]:
# Droping houses with Bedroom greater than 10
df_sale.drop(df_sale[df_sale['Bed']>10].index, inplace=True)

In [44]:
# Before working on houses with bedrooms equal to zero
df_sale[((df_sale['Bed']==0)|(df_sale['Bed'].isna())) & (df_sale['Description'].str.contains('bedroom'))].shape

(1297, 21)

In [14]:
df_sale.shape

(25670, 21)

In [45]:
def deep_cleaner(data):
    # Droping houses with Bedroom greater than 10
    data.drop(data[data['Bed']>10].index, inplace=True)
    
    # Working on houses with bedrooms equal to zero but information can be gotten from the description column.
    d1 = data[((data['Bed']==0)|(data['Bed'].isna())) & (data['Description'].str.contains('bedroom'))]
    
    # List of numbers of room gotten from description information
    mylist = [int(re.findall('\d+',(re.findall('\d+bedroom', x.replace(' ','')))[0])[0]) if len(re.findall('\d+bedroom', x.replace(' ',''))) > 0 else 0
        for x in d1['Description']]
    
    # Inserting values gotten from the description
    d1.loc[:, 'Bed']= mylist
    d1.loc[:, 'Bath']= mylist
    d1.loc[:, 'Toilet']=[y+1 if y!= 0 else y for y in mylist]
    
    #Assigning these values to the original Dataframe
    data[((data['Bed']==0)|(data['Bed'].isna()))  & (data['Description'].str.contains('bedroom'))] = d1
    
    # Assigning bath value to houses with bedroom == 0 or Nan. An intelligent guess.
    data[((data['Bed']==0)|(data['Bed'].isnull())) & (data['Bath']==1)] = data[((data['Bed']==0)|(data['Bed'].isnull())) & (data['Bath']==1)].assign(Bed = 1, inplace=True)
    data[((data['Bed']==0)|(data['Bed'].isnull())) & (data['Bath']==2)] = data[((data['Bed']==0)|(data['Bed'].isnull())) & (data['Bath']==2)].assign(Bed = 2, inplace=True)
    data[((data['Bed']==0)|(data['Bed'].isnull())) & (data['Bath']==3)] = data[((data['Bed']==0)|(data['Bed'].isnull())) & (data['Bath']==3)].assign(Bed = 3, inplace=True)
    
    # Filling Bed, Bath and Toilet nan values with 1 for Self contain being mini flats have 1 room.
    tmp = data[(data['Bed'].isnull()) & (data['Property Type'] == 'Self contain')]
    tmp = tmp.assign(Bed=1, Bath=1, Toilet=1, inplace=True)
    data[(data['Bed'].isnull()) & (data['Property Type'] == 'Self contain')] = tmp
    
    # Dropping rows with bed, bath and toilet value equal to zero
    data.drop(data[(data['Bed']==0)].index, inplace = True) 
    
    # Dropping Self Contain houses with bedrooms greater than 1
    data.drop(data[(data['Property Type'] == 'Self contain') & (data['Bed'] > 1)].index, inplace=True)
    
    # Dropping properties which are short lets
    data.drop(data[(data['Description'].str.contains('shortlet')) & (pd.notna(data['Description']))].index, inplace=True)
    
    # The rent Built or Serviced column has more than 50% missing data, so it should be dropped.
    data.drop('Built or Serviced', 1, inplace=True)
    
    # Filling Bed, Bath and Toilet nan values with 1 for mini flats being mini flats have 1 room.
    tmp = data[(data['Bed'].isnull()) & (data['Description'].str.contains('mini flat')|data['Description'].str.contains('self contain'))]
    tmp[['Bed','Bath','Toilet']] = tmp[['Bed','Bath','Toilet']].fillna(1)
    data[(data['Bed'].isnull()) & (data['Description'].str.contains('mini flat')|data['Description'].str.contains('self contain'))] = tmp
    
    tmp = data[(data['Bed'].isnull()) & (data['Property Type']=='Mini flat')]
    tmp[['Bed','Bath','Toilet']] = tmp[['Bed','Bath','Toilet']].fillna(1)
    data[(data['Bed'].isnull()) & (data['Property Type']=='Mini flat')] = tmp
    
    return data

In [20]:
df_sale2 = deep_cleaner(df_sale)

In [21]:
df_sale2.shape

(25268, 20)

In [46]:
pd.DataFrame({'Percentage_Mising':(df_sale2.isnull().sum()/df_sale2.shape[0])* 100})

,Percentage_Mising
Property,0.000000
Location,0.000000
Description,0.011873
Features,3.929872
Price,0.000000
Bed,1.242678
Bath,1.472218
Toilet,1.238721
Property Type,0.000000
Area,0.000000


In [47]:
# Filling Bed values with Bath values for rows that has bed as null
tmp = df_sale2[(df_sale2['Bed'].isnull()) & (df_sale2['Bath']>0)]
tmp['Bed'] = tmp['Bath']
df_sale2[(df_sale2['Bed'].isnull()) & (df_sale2['Bath']>0)] =tmp

In [48]:
# Assinging toilets with nan values with values from bed and adding 1 to it. An intelligent guess
tmp = df_sale2[(df_sale2['Toilet'].isnull()) & (pd.notna(df_sale2['Bed']))]
tmp['Toilet'] = tmp['Bed']+1
tmp['Bath'] = tmp['Bed']
df_sale2[(df_sale2['Toilet'].isnull()) & (pd.notna(df_sale2['Bed']))] = tmp

In [49]:
# Dropping the rest rows with missing bed, bath and toilet values
df_sale3 = df_sale2.dropna(subset=['Bed', 'Bath', 'Toilet'])

In [50]:
cleaned_data = df_sale3.drop(['Property','Location','Description','Features',],axis=1)

In [35]:
path


'C:\\Users\\HP\\Desktop\\Data Jedi\\new\\data_cleaned'

In [28]:
cleaned_data.to_csv(path+'/')

(24896, 16)

In [27]:
pd.DataFrame({'Percentage_Mising':(cleaned_data.isnull().sum()/cleaned_data.shape[0])* 100})

,Percentage_Mising
Price,0.0
Bed,0.0
Bath,0.0
Toilet,0.0
Property Type,0.0
Area,0.0
Purchase Type,0.0
Parking Space,0.0
Security,0.0
Electricity,0.0


In [51]:
## ENCODE DATA FOR PREDICTION ##
#One-Hot Encoding the Categorical Variables
ohe_object = OneHotCategoricalEncoder(top_categories=None,drop_last=True)
ohe_object.fit(cleaned_data)
tmp = ohe_object.transform(cleaned_data)

In [52]:
target = tmp['Price']
features = tmp.drop('Price',1)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=0)

In [54]:
model_lr = RandomForestRegressor()

In [55]:
model_lr.fit(X_train, y_train)

RandomForestRegressor()

In [56]:
model_lr.predict(X_test)

array([5.48795502e+07, 6.82958684e+07, 9.50758590e+07, ...,
       2.02169365e+08, 7.85555927e+06, 2.10583492e+08])

In [34]:
model_lr.score(X_train, y_train)

0.8611926297910892

In [ ]:
# Predictor Function based on location.
def predictor(data, area):
    data = data[data['Area'] == area.lower()]
    
    # One-Hot Encoding the Categorical Variables
    ohe_object = OneHotCategoricalEncoder(top_categories=None,drop_last=True)
    ohe_object.fit(data)
    
    # Encoding our data
    tmp = ohe_object.transform(data)
    
    # Assigning the Feature and Target values
    target = tmp['Price']
    features = tmp.drop('Price',1)
    
    # Splitting our data into Train and Test Sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=10)
    
    # Creating the algorithm class
    model_lr = RandomForestRegressor()
    
    # Creating algorithm object
    model_lr.fit(X_train, y_train)
    
    # Score of our algorithm
    score = model_lr.score(X_train, y_train)
    
    # Predicted values
    predict = model_lr.predict(X_train)
    
    # Creating DataFrame to compare predicted values
    predict_dataf = pd.DataFrame({'Predicted':predict, 'Real Values':y_train})
    
    return f'The R square score of the predictor is {score}'